In [2]:
import torch
import numpy as np
import warnings
import matplotlib.pyplot as plt

# pyTorch vs Numpy

### Конвертация из numpy в torsh и обратно

#### Пример 1 - Вектор

In [12]:
#Создадим простой вектор
vec = np.array([9,8,7])

In [13]:
vec

array([9, 8, 7])

In [14]:
vec.shape

(3,)

In [15]:
#Переведем вектор в torch
vec_tensor = torch.from_numpy(vec)

In [16]:
vec_tensor

tensor([9, 8, 7], dtype=torch.int32)

In [17]:
vec_tensor.shape

torch.Size([3])

In [18]:
vec_tensor.size()

torch.Size([3])

In [19]:
#Занулим тензор-вектор
vec_tensor -= vec_tensor
vec_tensor

tensor([0, 0, 0], dtype=torch.int32)

In [22]:
#вызовем исходный вектор из numpy
#он так же зануляется, важно это помнить
vec

array([0, 0, 0])

- ВАЖНО! Изменяя тензор, ты меняешь и исходник!

#### Пример 2 - Матрица

In [26]:
#создадим матрицу
matrix = np.array([[1,3,9],[5,7,3]])
matrix

array([[1, 3, 9],
       [5, 7, 3]])

In [28]:
matrix.shape

(2, 3)

In [31]:
#переведем ее в тензор, аналогично вектору
matrix_tensor = torch.from_numpy(matrix)
matrix_tensor

tensor([[1, 3, 9],
        [5, 7, 3]], dtype=torch.int32)

In [36]:
#создадим картинку
pic = torch.rand(3,224,224)
pic

tensor([[[0.1269, 0.6172, 0.6103,  ..., 0.9800, 0.3115, 0.4825],
         [0.6276, 0.4509, 0.9325,  ..., 0.7692, 0.9073, 0.7928],
         [0.9187, 0.0165, 0.3081,  ..., 0.2476, 0.7615, 0.6460],
         ...,
         [0.8247, 0.6875, 0.4594,  ..., 0.7486, 0.2351, 0.1341],
         [0.9465, 0.5525, 0.9754,  ..., 0.6821, 0.4893, 0.4236],
         [0.5098, 0.4143, 0.9372,  ..., 0.2492, 0.6413, 0.0831]],

        [[0.9883, 0.4773, 0.9992,  ..., 0.7255, 0.7487, 0.0913],
         [0.4857, 0.6997, 0.8745,  ..., 0.3947, 0.9065, 0.4082],
         [0.9040, 0.1325, 0.6632,  ..., 0.6700, 0.4610, 0.6634],
         ...,
         [0.5849, 0.3343, 0.3035,  ..., 0.5584, 0.7917, 0.9600],
         [0.8998, 0.6757, 0.9579,  ..., 0.0501, 0.7202, 0.0459],
         [0.0904, 0.9628, 0.4014,  ..., 0.8227, 0.1690, 0.3795]],

        [[0.3658, 0.9552, 0.4298,  ..., 0.9076, 0.3004, 0.6513],
         [0.9727, 0.5042, 0.1429,  ..., 0.3597, 0.6709, 0.2365],
         [0.2996, 0.2788, 0.3454,  ..., 0.1269, 0.7238, 0.

In [37]:
pic.shape

torch.Size([3, 224, 224])

In [38]:
#переведем матрицу обратно в numpy

In [39]:
matrix_tensor.numpy()

array([[1, 3, 9],
       [5, 7, 3]])

### Создание случайных Тензоров

In [40]:
torch.tensor(np.array([0,1.,2,3]))

tensor([0., 1., 2., 3.], dtype=torch.float64)

Сейчас мы рассмотрим основные способы создание тензоров. У torch все строже, чем у numpy, поэтому необходимо указывать типы данных при создании. Именно это мы сейчас и рассмотрим.

In [45]:
#создаем тензоры 32, 16 и 64 бита
print(
    torch.FloatTensor().dtype,\
    torch.HalfTensor().dtype,\
    torch.DoubleTensor().dtype, sep = '\n'
)

torch.float32
torch.float16
torch.float64


In [47]:
#рассмотрим целочисленные тензоры
print(
    torch.IntTensor().dtype,\
    torch.ShortTensor().dtype,\
    torch.LongTensor().dtype, sep = '\n'
)

torch.int32
torch.int16
torch.int64


- Научимся конвертировать типы

Это необходимо нам, чтобы менять данные, если они подаются не в том виде

In [49]:
#наша матрица
matrix_tensor.dtype

torch.int32

- первый способ - .type_as()

In [52]:
matrix_tensor = matrix_tensor.type_as(torch.FloatTensor())
matrix_tensor.dtype

torch.float32

- второй способ - .to()

In [58]:
#создадим новый тензор для наглядности
tensor_ = torch.tensor(np.array([0,1.,2,3]))

In [59]:
#Посмотрим его тип
tensor_.dtype

torch.float64

In [56]:
#Преобразуем
tensor_.to(torch.FloatTensor())

tensor([0., 1., 2., 3.])

In [60]:
#Проверим
tensor_.dtype

torch.float64

In [63]:
#Создаем рандомный тензор
torch.FloatTensor(42)

tensor([6.4805e-10, 6.3010e-10, 6.6376e-07, 6.8357e-04, 2.7098e-09, 1.0859e-05,
        6.4097e-10, 1.4580e-19, 1.1495e+24, 3.0956e-18, 3.7748e-08, 1.7585e-04,
        1.3443e+22, 1.7156e-07, 4.2531e-05, 2.7180e+23, 1.7079e-07, 2.1363e-07,
        3.0681e-18, 2.0552e+32, 1.8755e+28, 3.1093e-18, 2.0552e+32, 1.8755e+28,
        3.1093e-18, 1.2845e+31, 1.8395e+25, 6.1963e-04, 2.0768e+20, 2.6845e+23,
        2.1955e-04, 2.0778e+20, 3.3586e-06, 2.0892e+20, 1.0514e-05, 4.1300e-08,
        2.3076e-12, 1.8788e+31, 7.9303e+34, 6.1949e-04, 1.8590e+34, 7.7767e+31])

In [66]:
torch.FloatTensor(42).shape

torch.Size([42])

In [67]:
torch.ones(2,1)

tensor([[1.],
        [1.]])

In [68]:
torch.zeros(2,1)

tensor([[0.],
        [0.]])

### Инициализация из заданных распределений

In [70]:
x = torch.randn((4,2,4))
x

tensor([[[ 1.3643, -0.5046, -0.6052,  1.7805],
         [-0.0981, -0.5422, -0.1481,  0.8549]],

        [[ 0.0997, -0.5488,  1.1891, -0.8772],
         [-0.3948, -0.5683,  0.5613,  1.7160]],

        [[-2.1647, -1.0980, -0.4289, -0.2725],
         [ 0.9276, -1.1354, -0.6033,  0.9179]],

        [[-0.2740, -0.0926,  1.5912,  1.3064],
         [-0.0807, -0.2115, -0.0717,  0.0378]]])

In [71]:
x.size()

torch.Size([4, 2, 4])

In [72]:
#приведем наш тензор к равномерному распределению
x.uniform_(0,2)

tensor([[[1.5096, 0.3955, 0.0030, 1.9485],
         [0.6860, 0.4908, 0.9205, 1.4795]],

        [[1.8188, 1.2462, 1.7960, 0.0853],
         [0.3148, 0.0214, 0.9815, 1.2002]],

        [[1.5308, 0.9992, 0.6958, 0.1651],
         [0.2682, 1.5167, 0.1053, 1.6110]],

        [[0.1255, 0.4281, 0.2509, 1.5940],
         [1.2263, 1.0968, 0.2995, 0.2518]]])

- ВАЖНО! функция - .uniform_() имеет нижнее подчеркивание _. Такие функции производят изменения inplace

In [73]:
#ПРиведем к распределению бернулли
x.bernoulli_(p=0.5)

tensor([[[1., 1., 1., 1.],
         [0., 1., 1., 0.]],

        [[0., 1., 0., 1.],
         [0., 1., 1., 1.]],

        [[1., 1., 0., 0.],
         [0., 1., 0., 1.]],

        [[1., 0., 0., 1.],
         [0., 1., 0., 0.]]])

In [74]:
x.normal_(mean=0, std=1)

tensor([[[ 0.7006,  0.0585, -0.3159,  0.4419],
         [ 0.5647,  0.7964, -0.3778, -0.8843]],

        [[-0.2794, -0.1907,  0.4743, -2.4615],
         [ 0.7641, -0.7115, -0.2612,  1.1438]],

        [[-0.4735,  0.0412, -0.9434,  1.0634],
         [-0.6443, -0.6359, -0.1736,  1.0178]],

        [[-0.1418, -0.4358, -0.9070, -0.5860],
         [-1.7334, -0.9684, -1.7837,  0.1335]]])

### Основные операции над тензорами

In [75]:
#создадим два тензора

In [78]:
x = torch.FloatTensor([[1,1,1],[1,1,1]])
y = torch.FloatTensor([[-1,-1,-1],[-1,-1,-1]])
x, y

(tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[-1., -1., -1.],
         [-1., -1., -1.]]))

In [77]:
#сложение
x + y

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [84]:
x.add(y)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [85]:
# если мы поставим _ , то изменим наш x
x.add_(y)

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [86]:
x

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [88]:
#инициализируем снова наш x
x = torch.FloatTensor([[1,1,1],[1,1,1]])

In [89]:
#вычитание
x - y

tensor([[2., 2., 2.],
        [2., 2., 2.]])

In [92]:
x.sub(y)

tensor([[2., 2., 2.],
        [2., 2., 2.]])

In [90]:
#умножение
x * y

tensor([[-1., -1., -1.],
        [-1., -1., -1.]])

In [93]:
x.mul(y)

tensor([[-1., -1., -1.],
        [-1., -1., -1.]])

In [91]:
#деление
x / y

tensor([[-1., -1., -1.],
        [-1., -1., -1.]])

In [94]:
x.div(y)

tensor([[-1., -1., -1.],
        [-1., -1., -1.]])

- ко всем, вышеперечисленным операциям можно добавить _ и провести их inplace

In [107]:
x = torch.FloatTensor([[1,1,1],[1,1,2]])
y = torch.FloatTensor([[-1,1,1],[2,-1,-1]])

In [108]:
#сравнение
x > y

tensor([[ True, False, False],
        [False,  True,  True]])

In [109]:
x != y

tensor([[ True, False, False],
        [ True,  True,  True]])

In [110]:
x == y

tensor([[False,  True,  True],
        [False, False, False]])

- в последствии сравнения можно использовать как маску

In [111]:
mask = x > y

In [112]:
#Получаем только те позиции. которые больше
x[mask]

tensor([1., 1., 2.])

In [113]:
y.abs()

tensor([[1., 1., 1.],
        [2., 1., 1.]])

In [119]:
#Посмотрим на скалярное произведение
x = torch.FloatTensor([[1,2,3]])
y = torch.FloatTensor([[-1,2,-3]])

In [116]:
x @ y.t()

tensor([[-6.]])

In [118]:
x.mm(y.t())

tensor([[-6.]])

- Выводы: большинство операций, которые есть в Numpy, можно найти и в torch

### Использование агрегаций